In [14]:
from neo4j import GraphDatabase

user_id = 1110008753489760261
tweet_id = 1109874238398320640
politic_usa_id = 14792049

uri = 'bolt://localhost:7687'
driver = GraphDatabase.driver(uri)

In [15]:
def retrieve_all_tweet_stories():
    message = """MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {type: 'TWEET'})
                 WHERE NOT (start)-[:REPLIED_TO|RETWEETED_FROM]->() AND NOT ()-[:REPLIED_TO|RETWEETED_FROM]->(end)
                 WITH *, relationships(p) AS r
                 RETURN end.created_at, start.created_at, r"""
    records = driver.session().run(message).data()
    return records

In [16]:
def retrieve_tweet_story(tweet_id):
    message = """MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {type: 'TWEET'})
                 WHERE NOT (start)-[:REPLIED_TO|RETWEETED_FROM]->() AND NOT ()-[:REPLIED_TO|RETWEETED_FROM]->(end)
                 AND start.id = {id}
                 WITH *, relationships(p) AS r
                 RETURN end.created_at, start.created_at, r"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [17]:
def retrieve_users_who_interacted_at_a_story(tweet_id):
    message = """MATCH u=(user:User)-[:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet)<-[:TWEETED]-(storyStarter:User)
                 WHERE start.id = {id} AND (end.type = 'RETWEET' OR end.type = 'REPLY') AND start.type = 'TWEET'
                 RETURN user, storyStarter"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [18]:
def retrieve_stories_where_user_is_involved(user_id):
    message = """MATCH u=(user:User)-[:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet)<-[:TWEETED]-(storyStarter:User)
                 WHERE user.id = {id} or storyStarter.id = {id} AND (end.type = 'RETWEET' OR end.type = 'REPLY') AND start.type = 'TWEET'
                 WITH *, relationships(u) as re
                 RETURN end, re, start, user, storyStarter"""
    records = driver.session().run(message, id=user_id).data()
    return records

In [19]:
def retrieve_tweet_chain_ordered_by_time(tweet_id):
    # not needed, tweet chains are created in order in the first place
    print()

In [20]:
def retrieve_stories_made_by_user(user_id):
    message = """MATCH u=(user:User)-[:TWEETED]->(tweet:Tweet)
                 WHERE user.id = {id}
                 MATCH p=(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(tweet:Tweet)
                 WHERE end.type = 'RETWEET' OR end.type = 'REPLY' AND tweet.type = 'TWEET'
                 RETURN u"""
    records = driver.session().run(message, id=user_id).value()
    return records

In [21]:
def retrieve_time_spread_of_tweet_story(tweet_id):
    records = retrieve_tweet_story(tweet_id)[0]
    start_time = records['start.created_at']
    end_time = records['end.created_at']
    message = "Original tweet was posted on %s. \nLast available reaction was posted on %s." % (start_time, end_time)
    
    return message

In [22]:
def trial_and_error(tweet_id):
    message = """MATCH u=(user:User)-[r:POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: {id}, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
                 WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
                 RETURN user, end.created_at as timestamp"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [23]:
def get_users_from_tweet_story(tweet_id):
    message = """MATCH u=(user:User)-[r:POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: {id}, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
                 WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
                 RETURN user.screen_name as user, end.created_at as timestamp, storyStarter.screen_name as starter, start.created_at as origin"""
    records = driver.session().run(message, id=tweet_id).data()
    return records

In [24]:
def check_if_source_follows_target(source, target):
    message = "MATCH u=({screen_name: {source}})-[:FOLLOWS]->({screen_name: {target}}) RETURN u"
    records = driver.session().run(message, source=source, target=target).values()
    return records

In [25]:
if check_if_source_follows_target('erinbaby', 'politicususa'):
    print('hello')
print(check_if_source_follows_target('erinbaby', 'politicususa'))

hello
[[<Path start=<Node id=77671 labels={'User'} properties={'followers': 354, 'id': 1109874352479047680, 'screen_name': 'erinbaby', 'friends': 457}> end=<Node id=68621 labels={'User'} properties={'followers': 68790, 'id': 14792049, 'screen_name': 'politicususa', 'friends': 4015}> size=1>]]


In [27]:
# print(retrieve_tweet_story(1109874238398320640))
# print(retrieve_users_who_interacted_at_a_story(1109874238398320640))
# for value in retrieve_stories_made_by_user(politic_usa_id):
#     print(value)
#     print("HELLO DARKNESS WORLD")
#     print("\n")

# print(retrieve_time_spread_of_tweet_story(tweet_id))

# print(retrieve_users_who_interacted_at_a_story(tweet_id))
# print(trial_and_error(tweet_id))

listOfIds = [1109973027255664643,
    1109953086108381185,
    1109973771371376640,
    1109983287240474624,
    1109850542950531073,
    1109790735367200773,
    1109738825738117120,
    1109922557959000064,
    1109957082722250753,
    1109940939555065857,
    1110001365068251137,
    1109791857775661056,
    1109874238398320640,
    1109884141581918208]
for tw_id in listOfIds:
#     print('hello')
    results = get_users_from_tweet_story(tw_id)
    obj = {
            'user': results[0]['starter'],
            'timestamp': results[0]['origin'],
            'followers': []
        }
    listOfUsers = [obj]
    for record in results:
        obj = {
            'user': record['user'],
            'timestamp': record['timestamp'],
            'followers': []
        }
        listOfUsers.append(obj)

    # print(listOfUsers)
    for user in listOfUsers:
    #     followers = []
        for i in range(1,len(listOfUsers) - 1):
            if user['user'] != listOfUsers[i]['user'] and check_if_source_follows_target(listOfUsers[i]['user'], user['user']):
                if listOfUsers[i]['user'] not in user['followers']:
                    user['followers'].append(listOfUsers[i]['user'])
    #     record['followers'] = followers
    for record in listOfUsers:
        print(record)

{'user': 'BreitbartNews', 'timestamp': 'Mon Mar 25 00:19:26 EET 2019', 'followers': ['GFH_17', 'buckeye36', 'briandconnolly', 'HenryYearwood', 'dyerc2012', '487d341f306c4b3', 'taniece623', 'JimLoonie', 'RobertJ98986257', 'TrueloveWill', 'quantumshadow44', 'BeastOfWood', 'jackdten7172', 'bobby_dread', 'crabbydick', 'Buckzona', 'dbnicholls1', 'tweetwen115', 'JimPolk', 'KathyKomesar', 'bennettLee12', 'JoeAlex2344643', 'abeligbod', 'karinsabatini1', 'Belle3387', 'LeeHilden', 'JOgurek', 'shortman5427', 'Jholmes1968', 'FlybyNovel', 'calling12001', 'excalifsane', 'Shone27218200', 'SmallgovConserv', 'bfuenz14', 'Ana_love050', 'jdltek', 'DavidH692', 'tfedrick', '88reasonswhy', 'RedwardsRobin', 'jshindehite', 'interimspace', 'flyboy7798', 'rockie_springer', 'Tanya10049407', 'Unspinnable', 'GabyJuang', 'TousekE', 'Urireo561', 'KcajNehoc', 'MAGAcovfefe2020', 'mannreagan', 'ftman21', 'lissamac15', 'cloud_lillie', 'etmercado94', 'DannyMesser5', 'rag_koko', 'jpwilloughby', 'GayeRobinson18', 'concerne

{'user': 'ClickHole', 'timestamp': 'Sun Mar 24 23:00:12 EET 2019', 'followers': ['Famulator64', 'prismpiece', 'emilypressy', 'emmett_lui', 'AvatarJohnson', 'OkamiWolf10', 'KenziKardashian', 'OrganJaws', 'Towelprazolam', 'angry_yawns', 'CatherineExpat', 'sean_eick', 'DanSanchez1869']}
{'user': 'Famulator64', 'timestamp': 'Sun Mar 24 23:00:30 EET 2019', 'followers': []}
{'user': 'prismpiece', 'timestamp': 'Sun Mar 24 23:00:38 EET 2019', 'followers': []}
{'user': 'emilypressy', 'timestamp': 'Sun Mar 24 23:01:15 EET 2019', 'followers': []}
{'user': 'emmett_lui', 'timestamp': 'Sun Mar 24 23:02:01 EET 2019', 'followers': []}
{'user': 'AvatarJohnson', 'timestamp': 'Sun Mar 24 23:02:02 EET 2019', 'followers': []}
{'user': 'Vexillariuss', 'timestamp': 'Sun Mar 24 23:02:16 EET 2019', 'followers': []}
{'user': 'spoilerlesbians', 'timestamp': 'Sun Mar 24 23:02:32 EET 2019', 'followers': []}
{'user': 'OkamiWolf10', 'timestamp': 'Sun Mar 24 23:02:56 EET 2019', 'followers': []}
{'user': 'KenziKardash

{'user': 'politicususa', 'timestamp': 'Sun Mar 24 16:12:44 EET 2019', 'followers': ['silverthornn264', 'laleumen', 'ascalon5', 'BjayukCat', 'NetNewsBuzz', 'jimetoo', 'PROGRESSIVELIBE', 'cornerboy102000', 'PolitJunkieM']}
{'user': 'AndrewRei15', 'timestamp': 'Sun Mar 24 16:14:38 EET 2019', 'followers': []}
{'user': 'wyo2step', 'timestamp': 'Sun Mar 24 16:19:42 EET 2019', 'followers': ['DeniseKirchubel', 'RealTwitish', 'inthetao']}
{'user': '02catz', 'timestamp': 'Sun Mar 24 16:20:05 EET 2019', 'followers': []}
{'user': '02catz', 'timestamp': 'Sun Mar 24 16:20:20 EET 2019', 'followers': []}
{'user': 'MoreWhit', 'timestamp': 'Sun Mar 24 16:21:43 EET 2019', 'followers': []}
{'user': 'BluRidge4Bernie', 'timestamp': 'Sun Mar 24 16:22:43 EET 2019', 'followers': ['nancypeggy']}
{'user': 'BluRidge4Bernie', 'timestamp': 'Sun Mar 24 16:22:46 EET 2019', 'followers': ['nancypeggy']}
{'user': 'QuantumHumanity', 'timestamp': 'Sun Mar 24 16:28:27 EET 2019', 'followers': []}
{'user': 'ClaudineCaprio7',

{'user': 'politicususa', 'timestamp': 'Sun Mar 24 12:15:04 EET 2019', 'followers': ['Edgutk', 'StockScout1', 'vinkoview', 'genezog7', 'cld1205', 'silverthornn264', 'arcturus300', 'Mleeds2048', 'jimetoo']}
{'user': 'ghostdansing', 'timestamp': 'Sun Mar 24 12:16:25 EET 2019', 'followers': []}
{'user': 'jpcrowe_crowe', 'timestamp': 'Sun Mar 24 12:16:44 EET 2019', 'followers': []}
{'user': 'Edgutk', 'timestamp': 'Sun Mar 24 12:18:37 EET 2019', 'followers': ['vinkoview']}
{'user': 'keconlan', 'timestamp': 'Sun Mar 24 12:19:11 EET 2019', 'followers': []}
{'user': 'StockScout1', 'timestamp': 'Sun Mar 24 12:19:36 EET 2019', 'followers': []}
{'user': 'mhai56', 'timestamp': 'Sun Mar 24 12:19:52 EET 2019', 'followers': []}
{'user': 'cmoretti520', 'timestamp': 'Sun Mar 24 12:23:24 EET 2019', 'followers': []}
{'user': 'ColleenIrishLas', 'timestamp': 'Sun Mar 24 12:23:33 EET 2019', 'followers': []}
{'user': 'Indyria57Maria', 'timestamp': 'Sun Mar 24 12:23:49 EET 2019', 'followers': ['Betterw05759703

{'user': 'politicususa', 'timestamp': 'Sun Mar 24 23:16:05 EET 2019', 'followers': ['DarrellWest1984', 'riceor', 'TawdryOtter', 'chesse47', 'PROGRESSIVELIBE', 'BillSech', 'scott_squires', 'mikeedw56668977', 'KOANDMIKA', 'BrendaL03778178', 'Edgutk', 'Arriadna', 'SaudiaPaige', 'HenryBLowery', 'CrazyBob48', 'pamelaricks', 'BjayukCat', 'KDavCor', 'gabe_aguilar03', 'mcp_pol', 'RSmith1935', 'asutherland553', 'silverthornn264', 'BrendaHazelhur1', 'cld1205', 'Elljay1011', 'JohnnyHeldt', 'jimetoo', 'MerleJennings18', 'TassajaraRd', 'arcturus300', 'cmpnwtr', 'rjber15', 'PoliticusSarah', 'mentorinca', 'CitizenWonk', 'Mleeds2048', 'PolitJunkieM', 'jruha', 'robertfranek', 'pjRN', '4030lisa', 'wininmad', 'GGranma', 'goshofar', 'RiverCruiseChic', 'ascalon5', 'uo_duck99']}
{'user': 'DarrellWest1984', 'timestamp': 'Sun Mar 24 23:16:37 EET 2019', 'followers': []}
{'user': 'riceor', 'timestamp': 'Sun Mar 24 23:18:08 EET 2019', 'followers': ['CitizenWonk']}
{'user': 'TawdryOtter', 'timestamp': 'Sun Mar 24

{'user': 'politicususa', 'timestamp': 'Sun Mar 24 22:11:56 EET 2019', 'followers': ['AWorldOutOfMind', 'Lisa_Iannucci', 'POSUTtRUmp', 'BrendaL03778178', 'TrumpNewsDump', 'cosmicfirepeace', 'ConnieInAmerica', 'NikitaKitty', 'RobertMouton4', '27Seascapes', 'cmpnwtr', 'CliveLive2', 'NetNewsBuzz', 'kdv10s', 'syringa11', 'souzi54', 'rjber15', 'silverthornn264', 'evaleegifford11', 'princessearl1st', 'PolitJunkieM']}
{'user': 'AWorldOutOfMind', 'timestamp': 'Sun Mar 24 22:13:02 EET 2019', 'followers': []}
{'user': 'Lisa_Iannucci', 'timestamp': 'Sun Mar 24 22:13:08 EET 2019', 'followers': []}
{'user': 'POSUTtRUmp', 'timestamp': 'Sun Mar 24 22:13:37 EET 2019', 'followers': []}
{'user': 'Annealeeb', 'timestamp': 'Sun Mar 24 22:14:44 EET 2019', 'followers': []}
{'user': 'BrendaL03778178', 'timestamp': 'Sun Mar 24 22:15:03 EET 2019', 'followers': []}
{'user': 'TrumpNewsDump', 'timestamp': 'Sun Mar 24 22:16:24 EET 2019', 'followers': []}
{'user': 'cosmicfirepeace', 'timestamp': 'Sun Mar 24 22:18:30

{'user': '21WIRE', 'timestamp': 'Sun Mar 24 12:19:32 EET 2019', 'followers': ['e1af38f6dbad458', 'MurraySchwann', 'GeromanAT', 'kungfu_mandarin', 'hairymarx1', 'tngadd', 'wootendw', 'surfcityphoto', 'slink_E', 'IdonAkarnan', 'Beskooo', 'Mighty_Humanzee', 'Marwa__Osman', 'Sorgfelt', 'joaonunes67', 'ecocentrism1', 'kayalisalim', 'JohnMHenni2', 'squeeth', 'maurlind', 'Christi196155', 'Starsailoresss', 'dhillos318', 'JaredThreepoint', 'babstar', 'Meadows_Marin', 'HiwattPete', 'Souria4Syrians', 'freedomgirl2011', 'corruptbritain', 'petros_berhe', 'peter_donnelly', 'GrinchyPJ', 'aboutreed', 'dvernychuk', 'timesofiraq', 'MrLJones2', 'Jennife11439803', 'PhilipM67375789', 'gengh60', 'Alobhin', 'peterpobjecky', 'kazuma893', 'theaceofbass']}
{'user': 'e1af38f6dbad458', 'timestamp': 'Sun Mar 24 12:20:03 EET 2019', 'followers': ['Marwa__Osman']}
{'user': 'MurraySchwann', 'timestamp': 'Sun Mar 24 12:20:27 EET 2019', 'followers': []}
{'user': 'GeromanAT', 'timestamp': 'Sun Mar 24 12:21:09 EET 2019', 

{'user': 'politicususa', 'timestamp': 'Sun Mar 24 17:46:53 EET 2019', 'followers': ['erinbaby', 'MiamiNice57', 'silverthornn264', 'MarianMme16513', 'WhobbaBobba', 'Hattonsu', 'vinkoview', 'Mr_Scorpio', 'Bgolden53', 'MSMABAILEY', 'AwakenWholeness', 'maxthecat2000', 'MaryC_mlc4123', 'laleumen', 'syringa11', 'PolitiSmoose672', 'wininmad', 'evaleegifford11', 'genezog7', '7dnetwork', 'princessearl1st', 'LvnCali', 'mcp_pol', 'santo_aol', 'RBRB1516', 'maggievoelker', 'DianaSc99717236', 'Planelady1', 'StockScout1', 'FrenchieCat', 'wil91116385', 'gaelbill', 'chatta_girl', 'JohnnyHeldt', 'PolitJunkieM', 'cornerboy102000', 'purr10', 'barbara_kobrin', 'gramophoneman', 'Longley321']}
{'user': 'erinbaby', 'timestamp': 'Sun Mar 24 17:47:20 EET 2019', 'followers': []}
{'user': 'MiamiNice57', 'timestamp': 'Sun Mar 24 17:47:25 EET 2019', 'followers': ['sean_prevost']}
{'user': 'silverthornn264', 'timestamp': 'Sun Mar 24 17:48:18 EET 2019', 'followers': []}
{'user': 'MarianMme16513', 'timestamp': 'Sun Ma

In [28]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()

for user in listOfUsers:
    for follower in user['followers']:
        G.add_edge(user['user'], follower)
        
pos = nx.spring_layout(G)
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 500)
nx.draw_networkx_egdes(G, pos)
plt.show()

AttributeError: module 'networkx' has no attribute 'draw_networkx_egdes'

In [ ]:
# MATCH u=(user:User)-[r:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: 1109874238398320640, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
# WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
# WITH collect(user) as users, storyStarter, collect(end) as tweets, r, start
# UNWIND users AS user
# UNWIND tweets AS tweet
# MATCH pa=(someone:User)-[:POSTED_REPLY|POSTED_RETWEET]->(post:Tweet)-[:REPLIED_TO|RETWEETED_FROM]->(tweet)<-[:POSTED_REPLY|POSTED_RETWEET]-(user)
# WHERE post.created_at < tweet.created_at AND (exists((user)-[:FOLLOWS]->(someone)) OR exists((user)-[:FOLLOWS]->(storyStarter)))
# RETURN someone, user


In [ ]:
# MATCH u=(user:User)-[r:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: 1109874238398320640, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
# WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
# RETURN u


In [ ]:
# MATCH u=(user:User)-[r:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: 1109874238398320640, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
# WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
# WITH collect(user) as users, storyStarter, collect(end) as tweets, r
# RETURN users, storyStarter

In [ ]:
# MATCH u=(user:User)-[r:TWEETED|POSTED_REPLY|POSTED_RETWEET]->(end:Tweet)-[:REPLIED_TO|RETWEETED_FROM*]->(start:Tweet {id: 1109874238398320640, type: 'TWEET'})<-[:TWEETED]-(storyStarter:User)
# WHERE end.type = 'RETWEET' OR end.type = 'REPLY'
# WITH collect(user) as users, storyStarter, collect(end) as tweets, r, start
# RETURN tweets, start